# SageMaker script mode
> How to load SageMaker builtin Linear Learner model with Apache MXNet in Python.

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker, apache, mxnet, gluon]
- image: images/copied_from_nb/images/2022-07-07-sagemaker-script-mode.jpeg

![](images/2022-07-07-sagemaker-script-mode.jpeg)